## Convert the output files of TalkNet to .csv file

In [1]:
import pickle
import pandas as pd
import numpy as np

In [ ]:
# parameters
path='./demo/002/pywork/' # change the path (of TalkNet output files) here
file_output = './out.csv'

In [2]:
filename=['faces.pckl', 'scene.pckl', 'scores.pckl', 'tracks.pckl']
with open(path+filename[3], 'rb') as f:
    data = pickle.load(f)
with open(path+filename[2], 'rb') as f:
    scoredata = pickle.load(f)
with open(path+filename[1], 'rb') as f:
    scenedata = pickle.load(f)

In [42]:
# get scene_id (which is the start frame of each scene)
print(scenedata)
# print(len(scenedata))
scene_id=[]
for i in range(len(scenedata)):
    id = int(str(scenedata[i]).split(",")[0].split("=")[-1])
    scene_id.append(id)
print(scene_id)    

[(00:00:00.000 [frame=0, fps=25.000], 00:27:20.520 [frame=41013, fps=25.000]), (00:27:20.520 [frame=41013, fps=25.000], 00:30:33.880 [frame=45847, fps=25.000])]
[0, 41013]


In [51]:
d,df=[],[]
scene=-1
for i in range(len(data)):
    # if the start frame of a track == the start frame of a scene, the scene_id+=1
    print("track:",i,"start frame:",data[i]['track']['frame'][0])
    if scene+1 < len(scene_id):
#         print(scene_id[scene+1])
        if data[i]['track']['frame'][0] > scene_id[scene+1]:
            scene+=1
            print("scene is now", scene)
        
    d.append({'frame':data[i]['track']['frame'], 
              'scene_id':scene,
              'track_id':i,
              'ASDscore':np.append(0, scoredata[i]), # len(scoredata) = len(data)-1, so add 0 at the head
              'bbox_xmin':data[i]['track']['bbox'][:,0], 
              'bbox_ymin':data[i]['track']['bbox'][:,1], 
              'bbox_xmax':data[i]['track']['bbox'][:,2],
              'bbox_ymax':data[i]['track']['bbox'][:,3]})
    df.append(pd.DataFrame(data=d[i]))

# note: 'track_id' works like 'face_id', but it changes when 1) the scene change is detected, 2) the bbox IOU of previous frame and current frame is < 0.5 

track: 0 start frame: 5
scene is now 0
track: 1 start frame: 8
track: 2 start frame: 8
track: 3 start frame: 19182
track: 4 start frame: 41014
scene is now 1
track: 5 start frame: 41014


In [52]:
result = pd.concat(df)
result_sort=result.sort_values(by=["frame", "track_id"])
# print(result)
# print(result_sort)
result_sort.to_csv(file_output,index=False)

      frame  scene_id  track_id  ASDscore   bbox_xmin   bbox_ymin  \
0         5         0         0       0.0  900.346802   99.506721   
1         6         0         0      -1.2  899.307800   99.881485   
2         7         0         0      -1.2  898.599243   99.857483   
3         8         0         0      -1.3  898.367065  100.132935   
4         9         0         0      -0.9  898.750732  100.211151   
...     ...       ...       ...       ...         ...         ...   
4828  45842         1         5      -0.9  292.144989  242.426910   
4829  45843         1         5      -1.1  292.370880  243.091888   
4830  45844         1         5      -1.3  291.979736  244.478958   
4831  45845         1         5      -1.1  292.145660  245.177017   
4832  45846         1         5      -0.8  291.484253  246.277451   

        bbox_xmax   bbox_ymax  
0     1035.601685  270.375000  
1     1034.697144  270.694061  
2     1034.214111  270.299805  
3     1034.057495  270.290222  
4     1034.